# Only Ear/Face Training

In [1]:
import os
import numpy as np
import time

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Concatenate, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [3]:
tf.__version__

'2.3.0'

In [4]:
ROOT_DIR = os.path.abspath("../../")
DS_DIR = os.path.join(ROOT_DIR, "Datasets", "VGGFaceEar", "ear_augmented")
DS_train_DIR = os.path.join(DS_DIR, "train")
DS_val_DIR = os.path.join(DS_DIR, "val")
DS_test_DIR = os.path.join(DS_DIR, "test")
img_width, img_height = 224, 224
batch_size_ = 32

In [5]:
trdata = ImageDataGenerator(
    rescale=1./255.)
traindata = trdata.flow_from_directory(
    directory=DS_train_DIR,
    target_size=(img_width,img_height),
    batch_size=batch_size_,
    shuffle=True,
    class_mode='sparse')

vldata = ImageDataGenerator(
    rescale=1./255.)
valdata = vldata.flow_from_directory(
    directory=DS_val_DIR, 
    target_size=(img_width,img_height),
    batch_size=32,
    shuffle=True,
    class_mode='sparse')

tsdata = ImageDataGenerator(
    rescale=1./255.)
testdata = tsdata.flow_from_directory(
    directory=DS_test_DIR, 
    target_size=(img_width,img_height),
    batch_size=1,
    shuffle=False,
    class_mode='sparse',
)

Found 14868 images belonging to 25 classes.
Found 4032 images belonging to 25 classes.
Found 192 images belonging to 25 classes.


In [6]:
set_size = 25   #clases

### VGG16 model

In [7]:
cnn = VGG16(weights="imagenet", 
             include_top=False, 
             input_shape=(img_width, img_height, 3))

In [8]:
for layer in cnn.layers:
    layer.trainable = False
    

In [9]:
#cnn.summary()

In [10]:
#1 -> Droput 0.5 LR 0.001
#2 -> Dropout 0.3 LR 0.001
#3 -> without dropout LR 0.001
#4 -> Dropout 0.3 LR 0.0001
#5 -> Dropout 0.5 LR 0.0001
out = Flatten()(cnn.output)
out = (Dense(4096, activation='relu', name='fc1'))(out)
out = (Dropout(0.5))(out)
out = (Dense(4096, activation='relu', name='fc2'))(out)
out = (Dropout(0.5))(out)
out = (Dense(set_size, activation='softmax', name='predictions'))(out)

model = Model(inputs=[cnn.input], outputs=[out])
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [11]:
#model_face.compile(loss='sparse_categorical_crossentropy',
#             optimizer='sgd',
#             metrics=['accuracy'])

In [12]:
#model.compile(optimizer=Adam(lr=0.1), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])  #vgg16_1.h5--> 58%
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss=keras.losses.sparse_categorical_crossentropy, metrics=['sparse_categorical_accuracy'])  #vgg16_2.h5--> 99%  #with 2 dropout layers
#model_face.compile(optimizer=SGD(lr=0.1, momentum=0.9), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])  #vgg16_2.h5--> 99.37%  #without dropout layers
#model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])  #vgg16_2.h5--> 98.75%
#model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])  #vgg16_2.h5--> 55%
#model_face.compile(optimizer=RMSprop(lr=0.1), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])  #vgg16_3.h5--> 55%

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

tensorboardcb = keras.callbacks.TensorBoard(log_dir='Tensorboard_logs', histogram_freq=0, write_graph=True)
checkpoint = ModelCheckpoint(
    "vgg16_Ear_aug.h5", 
    monitor='val_sparse_categorical_accuracy', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto')

early = EarlyStopping(
    monitor='val_sparse_categorical_accuracy', 
    min_delta=0, 
    patience=15,
    verbose=1, 
    mode='auto')

hist = model.fit(
    x=traindata, 
    #steps_per_epoch=traindata.samples//batch_size_,
    validation_data= valdata, 
    epochs=50,
    callbacks=[checkpoint,early,tensorboardcb])

Epoch 1/50
  1/465 [..............................] - ETA: 0s - loss: 3.8622 - sparse_categorical_accuracy: 0.0000e+00WARNING:tensorflow:From C:\Users\Griss\anaconda3\envs\tf22-gpu\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
465/465 [==============================] - ETA: 0s - loss: 3.2673 - sparse_categorical_accuracy: 0.0651
Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.14087, saving model to vgg16_Ear_aug.h5
465/465 [==============================] - 59s 128ms/step - loss: 3.2673 - sparse_categorical_accuracy: 0.0651 - val_loss: 3.1019 - val_sparse_categorical_accuracy: 0.1409
Epoch 2/50
465/465 [==============================] - ETA: 0s - loss: 3.0537 - sparse_categorical_accuracy: 0.1174
Epoch 00002: val_sparse_categorical_accuracy improved from 0.14087 to 0.18180, saving 

Epoch 21/50
465/465 [==============================] - ETA: 0s - loss: 1.0413 - sparse_categorical_accuracy: 0.7203
Epoch 00021: val_sparse_categorical_accuracy did not improve from 0.49702
465/465 [==============================] - 55s 119ms/step - loss: 1.0413 - sparse_categorical_accuracy: 0.7203 - val_loss: 1.8463 - val_sparse_categorical_accuracy: 0.4888
Epoch 22/50
465/465 [==============================] - ETA: 0s - loss: 0.9853 - sparse_categorical_accuracy: 0.7360
Epoch 00022: val_sparse_categorical_accuracy improved from 0.49702 to 0.49975, saving model to vgg16_Ear_aug.h5
465/465 [==============================] - 58s 126ms/step - loss: 0.9853 - sparse_categorical_accuracy: 0.7360 - val_loss: 1.7930 - val_sparse_categorical_accuracy: 0.4998
Epoch 23/50
465/465 [==============================] - ETA: 0s - loss: 0.9336 - sparse_categorical_accuracy: 0.7485
Epoch 00023: val_sparse_categorical_accuracy improved from 0.49975 to 0.50670, saving model to vgg16_Ear_aug.h5
465/465 [=

### Evaluating before Fusion

#### FACE

In [20]:
results = model.evaluate(valdata)
print("test loss, test acc:", results)

126/126 [==============================] - 51s 403ms/step - loss: 1.2370 - sparse_categorical_accuracy: 0.7121
test loss, test acc: [1.2369836568832397, 0.7120535969734192]


In [22]:
results = model.evaluate(testdata)
print("test loss, test acc:", results)

192/192 [==============================] - 26s 138ms/step - loss: 1.2656 - sparse_categorical_accuracy: 0.7344
test loss, test acc: [1.2655636072158813, 0.734375]


#### EAR

In [14]:
results = model.evaluate(valdata)
print("test loss, test acc:", results)

126/126 [==============================] - 11s 84ms/step - loss: 1.6900 - sparse_categorical_accuracy: 0.5538
test loss, test acc: [1.6900315284729004, 0.5538194179534912]


In [15]:
results = model.evaluate(testdata)
print("test loss, test acc:", results)

192/192 [==============================] - 2s 9ms/step - loss: 1.6553 - sparse_categorical_accuracy: 0.6094
test loss, test acc: [1.6553372144699097, 0.609375]


# Fuse training

In [1]:
import os
import numpy as np
import time

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Concatenate, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [3]:
tf.__version__

'2.3.0'

In [4]:
ROOT_DIR = os.path.abspath("../../")
DS_DIR_EAR = os.path.join(ROOT_DIR, "Datasets", "VGGFaceEar", "ear_25class")
DS_DIR_FACE = os.path.join(ROOT_DIR, "Datasets", "VGGFaceEar", "face_25class")
img_width, img_height = 224, 224
batch_size_ = 32

In [5]:
def generate_generator_multiple(generator,dir1, dir2, batch_size,img_height,img_width):
    genX1 = generator.flow_from_directory(dir1,
                              target_size = (img_height,img_width),
                              class_mode = 'sparse',
                              batch_size = batch_size,
                              shuffle=False)
    
    genX2 = generator.flow_from_directory(dir2,
                              target_size = (img_height, img_width),
                              class_mode = 'sparse',
                              batch_size = batch_size, 
                              shuffle=False)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        yield [X1i[0], X2i[0]], X2i[1]  #

In [6]:
trdata = ImageDataGenerator(
    rescale=1./255.)
traindata_face_ear = generate_generator_multiple(
    generator = trdata,
    dir1 = os.path.join(DS_DIR_FACE, "train"),
    dir2 = os.path.join(DS_DIR_EAR, "train"),
    batch_size = batch_size_,
    img_height = img_height,
    img_width = img_width,
)

vldata = ImageDataGenerator(
    rescale=1./255.)
valdata_face_ear = generate_generator_multiple(
    generator = vldata,
    dir1 = os.path.join(DS_DIR_FACE, "val"),
    dir2 = os.path.join(DS_DIR_EAR, "val"),
    batch_size = batch_size_,
    img_height = img_height,
    img_width = img_width,
)

tsdata = ImageDataGenerator(
    rescale=1./255.)
testdata_face_ear = generate_generator_multiple(
    generator = tsdata,
    dir1 = os.path.join(DS_DIR_FACE, "test"),
    dir2 = os.path.join(DS_DIR_EAR, "test"),
    batch_size = batch_size_,
    img_height = img_height,
    img_width = img_width,
)

In [7]:
a = next(traindata_face_ear)
b = next(valdata_face_ear)
c = next(testdata_face_ear)

Found 708 images belonging to 25 classes.
Found 708 images belonging to 25 classes.
Found 192 images belonging to 25 classes.
Found 192 images belonging to 25 classes.
Found 192 images belonging to 25 classes.
Found 192 images belonging to 25 classes.


In [8]:
set_size = 25   #clases

## Set the model

In [9]:
model_face = load_model("TrainedModels/vgg16_Face_aug.h5")

In [10]:
for layer in model_face.layers:
    layer.trainable = False
    layer._name = layer._name + "_cnn_face"

In [11]:
out1 = model_face.layers[-4].output
#out1 = (BatchNormalization()(out1))
out1.shape

TensorShape([None, 4096])

In [12]:
model_ear = load_model("TrainedModels/vgg16_Ear_aug.h5")

In [13]:
for layer in model_ear.layers:
    layer.trainable = False
    layer._name = layer._name + "_cnn_ear"

In [14]:
out2 = model_ear.layers[-4].output
#out2 = (BatchNormalization()(out2))
out2.shape

TensorShape([None, 4096])

In [15]:
con = Concatenate()([out1, out2])

#out = Flatten()(con)
#out = (Dense(4096, activation='relu', name='fc1'))(con)
#out = (Dropout(0.3))(out)
out = (Dense(4096, activation='relu', name='fc2'))(con)
out = (Dropout(0.5))(out)
out = (Dense(set_size, activation='softmax', name='predictions'))(out)

model = Model(inputs=[model_face.input, model_ear.input], outputs=[out])

In [16]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1_cnn_face (InputLayer)   [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_1_cnn_ear (InputLayer)    [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1_cnn_face (Conv2D)  (None, 224, 224, 64) 1792        input_1_cnn_face[0][0]           
__________________________________________________________________________________________________
block1_conv1_cnn_ear (Conv2D)   (None, 224, 224, 64) 1792        input_1_cnn_ear[0][0]            
_______________________________________________________________________________________

In [17]:
#model.compile(optimizer=Adam(lr=0.1), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])  #vgg16_1.h5--> 58%
#model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss=keras.losses.sparse_categorical_crossentropy, metrics=['sparse_categorical_accuracy'])  #vgg16_2.h5--> 99%  #with 2 dropout layers
model.compile(optimizer=SGD(lr=0.001, decay=0.0002, momentum=0.9, nesterov=True), loss=keras.losses.sparse_categorical_crossentropy, metrics=['sparse_categorical_accuracy'])  #vgg16_2.h5--> 99%  #with 2 dropout layers

#model.compile(optimizer=SGD(lr=0.1, momentum=0.9), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])  #vgg16_2.h5--> 99.37%  #without dropout layers
#model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])  #vgg16_2.h5--> 98.75%
#model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])  #vgg16_2.h5--> 55%
#model.compile(optimizer=RMSprop(lr=0.1), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])  #vgg16_3.h5--> 55%

In [18]:
#model.fit([traindata_face, traindata_ear], [valdata_face, valdata_ear], epochs=10)

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
tensorboardcb = keras.callbacks.TensorBoard(log_dir='Tensorboard_logs/multi/', histogram_freq=0, write_graph=True)

checkpoint = ModelCheckpoint(
    "vgg16_Face_Ear.h5", 
    monitor='val_sparse_categorical_accuracy', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto')

early = EarlyStopping(
    monitor='val_sparse_categorical_accuracy', 
    min_delta=0, 
    patience=15,
    #if it doesn’t see any rise in validation accuracy in 25,  the model will stop 
    verbose=1, 
    mode='auto')

hist = model.fit(
    traindata_face_ear,
    epochs=50,
    steps_per_epoch=708//32,
    validation_data=valdata_face_ear,
    validation_steps = 192//32,
    callbacks=[checkpoint,early,tensorboardcb])


Epoch 1/50
 1/22 [>.............................] - ETA: 0s - loss: 3.1671 - sparse_categorical_accuracy: 0.1250WARNING:tensorflow:From C:\Users\Griss\anaconda3\envs\tf22-gpu\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
22/22 [==============================] - ETA: 0s - loss: 4.4963 - sparse_categorical_accuracy: 0.0074
Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.18229, saving model to vgg16_Face_Ear.h5
22/22 [==============================] - 9s 411ms/step - loss: 4.4963 - sparse_categorical_accuracy: 0.0074 - val_loss: 3.1295 - val_sparse_categorical_accuracy: 0.1823
Epoch 2/50
22/22 [==============================] - ETA: 0s - loss: 3.7855 - sparse_categorical_accuracy: 0.0170
Epoch 00002: val_sparse_categorical_accuracy improved from 0.18229 to 0.51562, saving model to vgg

Epoch 22/50
22/22 [==============================] - ETA: 0s - loss: 0.0748 - sparse_categorical_accuracy: 0.9970
Epoch 00022: val_sparse_categorical_accuracy did not improve from 0.81771
22/22 [==============================] - 5s 220ms/step - loss: 0.0748 - sparse_categorical_accuracy: 0.9970 - val_loss: 0.6762 - val_sparse_categorical_accuracy: 0.8073
Epoch 23/50
22/22 [==============================] - ETA: 0s - loss: 0.0821 - sparse_categorical_accuracy: 0.9956
Epoch 00023: val_sparse_categorical_accuracy did not improve from 0.81771
22/22 [==============================] - 5s 220ms/step - loss: 0.0821 - sparse_categorical_accuracy: 0.9956 - val_loss: 0.6840 - val_sparse_categorical_accuracy: 0.8125
Epoch 24/50
21/22 [===========================>..] - ETA: 0s - loss: 0.0798 - sparse_categorical_accuracy: 0.9985
Epoch 00024: val_sparse_categorical_accuracy did not improve from 0.81771
22/22 [==============================] - 5s 221ms/step - loss: 0.0795 - sparse_categorical_accurac

Epoch 45/50
22/22 [==============================] - ETA: 0s - loss: 0.0352 - sparse_categorical_accuracy: 1.0000
Epoch 00045: val_sparse_categorical_accuracy did not improve from 0.82812
22/22 [==============================] - 5s 222ms/step - loss: 0.0352 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.6250 - val_sparse_categorical_accuracy: 0.8281
Epoch 46/50
22/22 [==============================] - ETA: 0s - loss: 0.0335 - sparse_categorical_accuracy: 0.9985
Epoch 00046: val_sparse_categorical_accuracy did not improve from 0.82812
22/22 [==============================] - 5s 221ms/step - loss: 0.0335 - sparse_categorical_accuracy: 0.9985 - val_loss: 0.6194 - val_sparse_categorical_accuracy: 0.8177
Epoch 47/50
21/22 [===========================>..] - ETA: 0s - loss: 0.0335 - sparse_categorical_accuracy: 1.0000
Epoch 00047: val_sparse_categorical_accuracy did not improve from 0.82812
22/22 [==============================] - 5s 221ms/step - loss: 0.0334 - sparse_categorical_accurac

In [20]:
results = model.evaluate(valdata_face_ear, steps= 192)
print("test loss, test acc:", results)

192/192 [==============================] - 34s 179ms/step - loss: 0.6296 - sparse_categorical_accuracy: 0.8229
test loss, test acc: [0.6295827031135559, 0.8229166865348816]


In [21]:
results = model.evaluate(testdata_face_ear, steps= 192)
print("test loss, test acc:", results)

192/192 [==============================] - 34s 179ms/step - loss: 0.6484 - sparse_categorical_accuracy: 0.8021
test loss, test acc: [0.6484200358390808, 0.8020833134651184]
